# Автокодировщик

In [ ]:
import time

import numpy as np
import theano
import theano.tensor as T
import lasagne

import matplotlib.pylab as plt
from utils import load_dataset, iterate_minibatches
%matplotlib inline

BATCH_SIZE = 20
HIDDEN_DIM = 2

num_epochs = 128

X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()

## Обучение модели

tl;dr: Автокодировщик может быть использован для построения маломерных признаков данных без разметки.

В процессе обучения строится пара отображений $E: \mathbb R^D \rightarrow R^d$ (кодировщик) и $D: \mathbb R^d \rightarrow R^D$ (декодировщик), чья композиция приближает тождественное отображение:

$$ D(E(x)) \approx x $$

In [ ]:
# Определим кодировщик и декодировщик с помощью пары полносвязных нейронных сетей

def ae_encoder(input_var):
    l_in = lasagne.layers.InputLayer(shape=(None, 1, 28, 28), input_var=input_var)
    ######################################################################################
    # Реализуйте некоторую несложную архитектуру кодировщика, возвращающую HIDDEN_DIM-мерный код #
    # Какие функции активации можно поставить на выход сети?                                  #
    ######################################################################################
    l_hid1 = lasagne.layers.DenseLayer(
            l_in, num_units=128,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(),
            name='e_hid1')
    l_hid2 = lasagne.layers.DenseLayer(
            l_hid1, num_units=64,
            nonlinearity=lasagne.nonlinearities.rectify,
            name='e_hid2')
    l_out = lasagne.layers.DenseLayer(
            l_hid2, num_units=HIDDEN_DIM,
            nonlinearity=None,
            name='e_out')
    return l_out


def ae_decoder(input_var):
    l_in = lasagne.layers.InputLayer(shape=(None, HIDDEN_DIM), input_var=input_var)
    ##################################################################################################
    # Реализуйте некоторую несложную архитектуру декодировщика, возвращающую батч объектов размера (1, 28, 28) #
    ##################################################################################################
    l_hid1 = lasagne.layers.DenseLayer(
            l_in, num_units=64,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(),
            name='d_hid1')
    l_hid2 = lasagne.layers.DenseLayer(
            l_hid1, num_units=128,
            nonlinearity=lasagne.nonlinearities.rectify,
            name='d_hid2')
    l_out = lasagne.layers.DenseLayer(
            l_hid2, num_units=28 * 28,
            nonlinearity=lasagne.nonlinearities.sigmoid,
            name='d_out')
    l_out = lasagne.layers.reshape(l_out, shape=(-1, 1, 28, 28))
    return l_out

In [ ]:
# Инициализируем сеть
input_x = T.tensor4('input_x')
    
encoder = ae_encoder(input_x)
decoder = ae_decoder(
    lasagne.layers.get_output(encoder)
)

Для обучения автокодировщика будем использовать среднеквадратичную ошибку

$$ L(X) = \frac{1}{N}\sum_{i=1}^{N} \sum_{j=1}^{28^2} \left( D(E(x_i))_j - x_{i,j} \right)^2 = \frac{1}{N}\sum_{i=1}^{N} (D(E(x_i)) - x_i)^T (D(E(x_i)) - x_i) $$

In [ ]:
#####################################################################################
# Определите операцию для вычисления функции потерь, а также создайте список параметров модели #
# для передачи в оптимизатор                                                           #
loss = lasagne.objectives.squared_error(
    lasagne.layers.get_output(decoder), input_x
).sum(axis=(1, 2, 3)).mean()
params = lasagne.layers.get_all_params([encoder, decoder])
#####################################################################################

updates = lasagne.updates.adam(loss, params)
 
train = theano.function(
    [input_x],
    loss,
    updates=updates
)
test_loss = theano.function(
    [input_x],
    loss
)

Обучение, как и во многих других случаях, выполяется с помощью стохастического градиентного спуска

In [ ]:
for epoch in range(num_epochs):
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, batchsize=BATCH_SIZE):
        train_err += train(batch)
        train_batches += 1
        
    test_err = 0
    test_batches = 0
    for batch in iterate_minibatches(X_test, batchsize=BATCH_SIZE):
        test_err += test_loss(batch)
        test_batches += 1
        
    print("Epoch {} of {} took {:.3f}s".format(
          epoch + 1, num_epochs, time.time() - start_time))
    print("Train error {}".format(train_err/train_batches))
    print("Test error {}".format(test_err/test_batches))

## Визуализация

Модель с двумерными скрытыми переменными легко визуализировать. Определим две функции: одну для построения пропущенных через автокодировщик изображений, вторую для вычисления скрытых представлений по изображению

In [ ]:
from utils import plot_reconstructions, plot_hidden_space

reconstruct = theano.function(
        [input_x],
        lasagne.layers.get_output(decoder)
)

encode = theano.function(
        [input_x],
        lasagne.layers.get_output(encoder)
)

Примеры изображений, пропущенных через автокодировщик: 

In [ ]:
plot_reconstructions(X_test, reconstruct)

Визуализация признакового пространства. Насколько пространство простое? Везде ли оно плотно? Как выбрать точку в этом пространстве, которая будет соответствовать коду какого-то объекта?

In [ ]:
plot_hidden_space(X_test[:1000], encode)

Попробуйте погенерировать изображения по паре координат

In [ ]:
input_z = T.matrix('input_z')

decode_a_code = theano.function(
    [input_z],
    lasagne.layers.get_output(decoder, input_z),
)

def generate_from_code(x, y):
    img = decode_a_code([[x, y]]).reshape((28, 28))
    plt.imshow(img, 'gray')

In [ ]:
generate_from_code(50., 20.)